# Tutorial: CNN with Pytorch using Opp dataset

In [1]:
import sys
sys.path.append('../')

In [2]:
# general
import numpy as np
import pandas as pd
from progressbar import ProgressBar

# dataset
from actreco.datasets.opportunityc import Opportunity
from actreco.sampling import sampling

# modeling
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


In [23]:
def choice_func(train=True, initial_test_state=0):
    i = [0]  # nb call
    def choice():
        i[0] += 1
        if train is True:
            return np.random.choice
        
        r = np.random.RandomState(initial_test_state+i[0])
        return r.choice
    return choice()


def batch_generator(x_list, y_list, batch_size, l_sample, train=True, nb_iter=1, categorical=False, seed=0):
    choice = choice_func(train, initial_test_state=seed)
    
    nb_sample = sum([x.shape[0] for x in x_list])
    idx_set = set(range(nb_sample))
    nb_seen_sample = 0
    for x in x_list:
        nb_seen_sample += x.shape[0]
        idx_set -= set(range(nb_seen_sample-l_sample+1, nb_seen_sample+1))
    valid_idx = list(idx_set)
    
    x = np.concatenate(x_list)
    y = np.concatenate(y_list)
    
    for i in range(nb_iter):
        start = choice(valid_idx, batch_size)
        X = sampling(x, 'clips', dtype='np', start=start, l_sample=l_sample)[..., np.newaxis].swapaxes(1, 3)
        Y = sampling(y, 'clips', dtype='np', start=start, l_sample=l_sample)
        Y = np.concatenate([Y.sum(axis=1), np.ones((batch_size, 1))], axis=-1).argmax(axis=-1)
        if categorical is not False:
            Y = np.eye(categorical)[Y]
        yield (X, Y)

In [5]:
# dataset

In [6]:
train_set = Opportunity(userID='S1,S2,S3')
x_train_list = train_set.get('X')
y_train_list = train_set.get('y')

In [7]:
test_set = Opportunity(userID='S4')
x_test_list = test_set.get('X')
y_test_list = test_set.get('y')

In [8]:
class VanilaCNN(nn.Module):
    def __init__(self, l_sample, nb_modal, nb_in_filter, nb_filter_list, kw, nb_unit, nb_out):
        super(VanilaCNN, self).__init__()
        self.l_sample = l_sample
        self.nb_modal = nb_modal
        self.nb_conv = len(nb_filter_list)
        
        if isinstance(kw, int):
            kw = [kw] * self.nb_conv
        assert len(nb_filter_list) == len(kw)
        nb_filter_list = [nb_in_filter] + nb_filter_list
        
        self.kw = kw
        self.nb_filter_list = nb_filter_list

        convs = []
        for i in range(self.nb_conv):
            convs.append(nn.Conv2d(nb_filter_list[i], nb_filter_list[i+1], (1, kw[i])))
        self.convs = nn.ModuleList(convs)
        self.fc1 = nn.Linear(self.conv_output_shape(), nb_unit)
        self.fc2 = nn.Linear(nb_unit, nb_out)
    
    def forward(self, x):
        for conv in self.convs:
            x = F.max_pool2d(F.relu(conv(x)), (1, 2))
            dropout = nn.Dropout2d(0.5)
            # x = dropout(x)
        x = x.view(-1, self.conv_output_shape())
        x = F.dropout(self.fc1(x), 0.5)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
    def conv_output_shape(self):
        length = self.l_sample
        for i in range(self.nb_conv):
            length = int((length - self.kw[i] + 1) /2)
        return length * self.nb_filter_list[-1] * self.nb_modal

In [9]:
nb_modal_dim = 2
nb_modal = train_set.nb_modal
nb_out = train_set.nb_class

kw = 3
nb_filter_list = [50, 40, 20]
nb_in_filter = 1
drop_rate = 0.5
nb_unit = 400

batch_size = 128
l_sample = 30
interval = int(l_sample) * 0.5

nb_iter = 100
report_each = 10

model = VanilaCNN(l_sample=l_sample, nb_modal=nb_modal, nb_in_filter=1, nb_filter_list=nb_filter_list, kw=kw, nb_unit=nb_unit, nb_out=nb_out).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

In [10]:
model.train()

p = ProgressBar(max_value=nb_iter)
gen = batch_generator(x_train_list, y_train_list, batch_size, l_sample, train=True, nb_iter=nb_iter, categorical=False)
for batch_idx, (X, y) in enumerate(gen):
    batch_idx += 1
    tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
    ty = Variable(torch.LongTensor(y), requires_grad=False).cuda()
    
    optimizer.zero_grad()
    output = model(tX)
    
    loss = criterion(output, ty)
    loss.backward()
    optimizer.step()
    if batch_idx % report_each == 0:
        print("{} samples, {} batch \t Loss: {:.6f}".format(batch_idx * batch_size, batch_idx, loss.data[0]))
    p.update(batch_idx)
    

 13% (13 of 100) |###                     | Elapsed Time: 0:00:00 ETA:  0:00:04

1280 samples, 10 batch 	 Loss: 2.420109


 25% (25 of 100) |######                  | Elapsed Time: 0:00:01 ETA:  0:00:03

2560 samples, 20 batch 	 Loss: 1.739100


 35% (35 of 100) |########                | Elapsed Time: 0:00:01 ETA:  0:00:02

...

10240 samples, 80 batch 	 Loss: 1.649884


 94% (94 of 100) |######################  | Elapsed Time: 0:00:04 ETA:  0:00:00

11520 samples, 90 batch 	 Loss: 1.794142


 99% (99 of 100) |####################### | Elapsed Time: 0:00:04 ETA:  0:00:00

12800 samples, 100 batch 	 Loss: 1.670719


In [11]:
from sklearn import metrics

def test(model, x_list, y_list, l_sample, nb_iter=1, batch_size=2048):
    model.eval()
    test_gen = batch_generator(x_train_list, y_train_list, batch_size, l_sample, train=False, nb_iter=nb_iter, categorical=False, seed=0)
    y_list = []
    py_list = []
    for X, y in test_gen:
        tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
        py = model(tX).max(1)[1].data.cpu().numpy().reshape(-1)
        y_list.append(y)
        py_list.append(py)
    
    y = np.concatenate(y_list)
    py = np.concatenate(py_list)
    print(metrics.accuracy_score(y, py))
    print(metrics.f1_score(y, py, average='macro'))
    print(metrics.precision_score(y, py, average=None))
    print(metrics.recall_score(y, py, average=None))

In [14]:
test(model, x_test_list, y_test_list, l_sample, nb_iter=1, batch_size=batch_size)

0.546875
0.0415923945336
[ 0.        0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.        0.
  0.546875]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


In [15]:
# keras model for comparison
# which is much slower than pytorch

In [25]:
import keras
if keras.__version__ >= '2.0.0':

    from keras.models import Sequential
    from keras.layers import Dense, Convolution2D, Dropout, Activation, Flatten
    from keras.layers import MaxPooling2D

    from keras.optimizers import SGD

    model = Sequential()
    model.add(Convolution2D(50, kernel_size=(1, 3), input_shape=[1, nb_modal, l_sample]))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(1, 2)))
    model.add(Convolution2D(40, (1, 3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(1, 2)))
    model.add(Convolution2D(20, (1, 3)))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(1, 2)))
    model.add(Flatten())
    model.add(Dense(nb_unit))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_out))
    model.add(Activation('softmax'))
    model.summary()

    optimizer = SGD(lr=0.01, momentum=0.5)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    
    p = ProgressBar(max_value=report_each)
    gen = batch_generator(x_train_list, y_train_list, batch_size, l_sample, train=True, nb_iter=nb_iter, categorical=train_set.nb_class)
    for batch_idx, (X, y) in enumerate(gen):
        batch_idx += 1
        loss = model.train_on_batch(X, y)
        if batch_idx % report_each == 0:
            print("Seen {} samples, {} batch \t Loss: {:.6f}".format(batch_idx * batch_size, batch_idx, float(loss)))
            p.update(report_each)
        else:
            p.update((batch_idx % report_each))
else:
    print("TBA")

TBA
